# Human-in-the-Loop-Workflow mit Microsoft Agent Framework

## 🎯 Lernziele

In diesem Notebook lernen Sie, wie Sie **Human-in-the-Loop**-Workflows mit dem `RequestInfoExecutor` des Microsoft Agent Frameworks implementieren. Dieses leistungsstarke Muster ermöglicht es, KI-Workflows zu unterbrechen, um menschliche Eingaben zu sammeln, wodurch Ihre Agenten interaktiv werden und Menschen die Kontrolle über kritische Entscheidungen erhalten.

## 🔄 Was ist Human-in-the-Loop?

**Human-in-the-loop (HITL)** ist ein Designmuster, bei dem KI-Agenten die Ausführung unterbrechen, um menschliche Eingaben anzufordern, bevor sie fortfahren. Dies ist besonders wichtig für:

- ✅ **Kritische Entscheidungen** - Holen Sie sich menschliche Zustimmung, bevor wichtige Aktionen durchgeführt werden
- ✅ **Mehrdeutige Situationen** - Lassen Sie Menschen klären, wenn die KI unsicher ist
- ✅ **Benutzervorlieben** - Bitten Sie Benutzer, zwischen mehreren Optionen zu wählen
- ✅ **Compliance & Sicherheit** - Stellen Sie menschliche Aufsicht für regulierte Vorgänge sicher
- ✅ **Interaktive Erlebnisse** - Erstellen Sie Konversationsagenten, die auf Benutzereingaben reagieren

## 🏗️ Wie funktioniert es im Microsoft Agent Framework?

Das Framework bietet drei Hauptkomponenten für HITL:

1. **`RequestInfoExecutor`** - Ein spezieller Executor, der den Workflow unterbricht und ein `RequestInfoEvent` auslöst
2. **`RequestInfoMessage`** - Basisklasse für typisierte Anforderungs-Payloads, die an Menschen gesendet werden
3. **`RequestResponse`** - Verknüpft menschliche Antworten mit den ursprünglichen Anfragen mithilfe von `request_id`

**Workflow-Muster:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Unser Beispiel: Hotelbuchung mit Benutzerbestätigung

Wir bauen auf dem bedingten Workflow auf, indem wir eine menschliche Bestätigung **vor** dem Vorschlagen alternativer Reiseziele hinzufügen:

1. Der Benutzer fordert ein Reiseziel an (z. B. "Paris")
2. Der `availability_agent` prüft, ob Zimmer verfügbar sind
3. **Falls keine Zimmer verfügbar sind** → Der `confirmation_agent` fragt: "Möchten Sie Alternativen sehen?"
4. Der Workflow **pausiert** mithilfe von `RequestInfoExecutor`
5. **Mensch antwortet** mit "ja" oder "nein" über die Konsoleneingabe
6. Der `decision_manager` leitet basierend auf der Antwort weiter:
   - **Ja** → Zeige alternative Reiseziele
   - **Nein** → Buchungsanfrage abbrechen
7. Zeige das endgültige Ergebnis an

Dies zeigt, wie Sie den Benutzern Kontrolle über die Vorschläge des Agenten geben können!

---

Legen wir los! 🚀


## Schritt 1: Erforderliche Bibliotheken importieren

Wir importieren die Standardkomponenten des Agent Frameworks sowie **spezifische Klassen für menschliche Eingaben**:
- `RequestInfoExecutor` - Executor, der den Workflow für menschliche Eingaben pausiert
- `RequestInfoEvent` - Ereignis, das ausgelöst wird, wenn menschliche Eingaben angefordert werden
- `RequestInfoMessage` - Basisklasse für typisierte Anforderungsdaten
- `RequestResponse` - Verknüpft menschliche Antworten mit Anfragen
- `WorkflowOutputEvent` - Ereignis zur Erkennung von Workflow-Ausgaben


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Schritt 2: Pydantic-Modelle für strukturierte Ausgaben definieren

Diese Modelle definieren das **Schema**, das von den Agenten zurückgegeben wird. Wir behalten alle Modelle aus dem bedingten Workflow bei und fügen hinzu:

**Neu für Human-in-the-Loop:**
- `HumanFeedbackRequest` - Unterklasse von `RequestInfoMessage`, die die Anfrage-Payload definiert, die an Menschen gesendet wird
  - Enthält `prompt` (Frage, die gestellt werden soll) und `destination` (Kontext über die nicht verfügbare Stadt)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Schritt 3: Erstellen des Hotelbuchungs-Tools

Dasselbe Tool wie im bedingten Workflow – überprüft, ob Zimmer am Zielort verfügbar sind.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Schritt 4: Bedingungsfunktionen für das Routing definieren

Wir benötigen **vier Bedingungsfunktionen** für unseren Human-in-the-Loop-Workflow:

**Aus dem bedingten Workflow:**
1. `has_availability_condition` - Leitet weiter, wenn Hotels VERFÜGBAR sind
2. `no_availability_condition` - Leitet weiter, wenn Hotels NICHT VERFÜGBAR sind

**Neu für Human-in-the-Loop:**
3. `user_wants_alternatives_condition` - Leitet weiter, wenn der Benutzer "Ja" zu Alternativen sagt
4. `user_declines_alternatives_condition` - Leitet weiter, wenn der Benutzer "Nein" zu Alternativen sagt


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Schritt 5: Erstellen des Decision Manager Executor

Das ist der **Kern des Human-in-the-Loop-Musters**! Der `DecisionManager` ist ein benutzerdefinierter `Executor`, der:

1. **Menschliches Feedback empfängt** über `RequestResponse`-Objekte
2. **Die Entscheidung des Nutzers verarbeitet** (ja/nein)
3. **Den Workflow steuert**, indem Nachrichten an die entsprechenden Agenten gesendet werden

Wichtige Merkmale:
- Verwendet den `@handler`-Dekorator, um Methoden als Workflow-Schritte verfügbar zu machen
- Empfängt `RequestResponse[HumanFeedbackRequest, str]`, das sowohl die ursprüngliche Anfrage als auch die Antwort des Nutzers enthält
- Gibt einfache "ja"- oder "nein"-Nachrichten aus, die unsere Bedingungsfunktionen auslösen


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Schritt 6: Erstellen eines benutzerdefinierten Anzeige-Executors

Der gleiche Anzeige-Executor aus dem bedingten Workflow - liefert die endgültigen Ergebnisse als Workflow-Ausgabe.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Schritt 7: Umgebungsvariablen laden

Konfigurieren Sie den LLM-Client (GitHub Models, Azure OpenAI oder OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Schritt 8: Erstellen von KI-Agenten und Ausführern

Wir erstellen **sechs Workflow-Komponenten**:

**Agenten (eingebettet in AgentExecutor):**
1. **availability_agent** - Überprüft die Hotelverfügbarkeit mit dem Tool
2. **confirmation_agent** - 🆕 Bereitet die Anfrage zur Bestätigung durch den Menschen vor
3. **alternative_agent** - Schlägt alternative Städte vor (wenn der Benutzer zustimmt)
4. **booking_agent** - Ermutigt zur Buchung (wenn Zimmer verfügbar sind)
5. **cancellation_agent** - 🆕 Bearbeitet die Stornierungsnachricht (wenn der Benutzer ablehnt)

**Spezielle Ausführer:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, der den Workflow für menschliche Eingaben pausiert
7. **decision_manager** - 🆕 Benutzerdefinierter Ausführer, der basierend auf der menschlichen Antwort weiterleitet (bereits oben definiert)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Schritt 9: Erstellen des Workflows mit Mensch-in-der-Schleife

Jetzt konstruieren wir den Workflow-Graphen mit **bedingter Weiterleitung**, einschließlich des Mensch-in-der-Schleife-Pfads:

**Workflow-Struktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Wichtige Verbindungen:**
- `availability_agent → confirmation_agent` (wenn keine Zimmer verfügbar sind)
- `confirmation_agent → prepare_human_request` (Typ umwandeln)
- `prepare_human_request → request_info_executor` (Pause für menschliche Eingabe)
- `request_info_executor → decision_manager` (immer - liefert RequestResponse)
- `decision_manager → alternative_agent` (wenn der Benutzer "ja" sagt)
- `decision_manager → cancellation_agent` (wenn der Benutzer "nein" sagt)
- `availability_agent → booking_agent` (wenn Zimmer verfügbar sind)
- Alle Pfade enden bei `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Schritt 10: Testfall 1 ausführen - Stadt OHNE Verfügbarkeit (Paris mit menschlicher Bestätigung)

Dieser Test zeigt den **vollständigen Mensch-im-Kreislauf-Zyklus**:

1. Anfrage nach einem Hotel in Paris
2. availability_agent prüft → Keine Zimmer verfügbar
3. confirmation_agent erstellt eine Frage für den Menschen
4. request_info_executor **pausiert den Workflow** und löst `RequestInfoEvent` aus
5. **Anwendung erkennt Ereignis und fordert Benutzer über die Konsole auf**
6. Benutzer gibt "ja" oder "nein" ein
7. Anwendung sendet Antwort über `send_responses_streaming()`
8. decision_manager leitet basierend auf der Antwort weiter
9. Endergebnis wird angezeigt

**Wichtiges Muster:**
- Verwenden Sie `workflow.run_stream()` für die erste Iteration
- Verwenden Sie `workflow.send_responses_streaming(pending_responses)` für nachfolgende Iterationen
- Hören Sie auf `RequestInfoEvent`, um zu erkennen, wann menschliche Eingabe erforderlich ist
- Hören Sie auf `WorkflowOutputEvent`, um die endgültigen Ergebnisse zu erfassen


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Schritt 11: Führen Sie Testfall 2 aus - Stadt MIT Verfügbarkeit (Stockholm - Keine menschliche Eingabe erforderlich)

Dieser Test zeigt den **direkten Ablauf**, wenn Zimmer verfügbar sind:

1. Hotel in Stockholm anfragen
2. availability_agent prüft → Zimmer verfügbar ✅
3. booking_agent schlägt Buchung vor
4. display_result zeigt Bestätigung
5. **Keine menschliche Eingabe erforderlich!**

Der Ablauf umgeht den menschlichen Eingriff vollständig, wenn Zimmer verfügbar sind.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Wichtige Erkenntnisse und Best Practices für Mensch-in-der-Schleife

### ✅ Was Sie gelernt haben:

#### 1. **RequestInfoExecutor-Muster**
Das Mensch-in-der-Schleife-Muster im Microsoft Agent Framework verwendet drei Hauptkomponenten:
- `RequestInfoExecutor` - Pausiert den Workflow und sendet Ereignisse
- `RequestInfoMessage` - Basisklasse für typisierte Anforderungs-Payloads (unterklassen!)
- `RequestResponse` - Verknüpft menschliche Antworten mit ursprünglichen Anfragen

**Wichtige Erkenntnis:**
- `RequestInfoExecutor` sammelt keine Eingaben selbst - es pausiert nur den Workflow
- Ihr Anwendungscode muss auf `RequestInfoEvent` hören und Eingaben sammeln
- Sie müssen `send_responses_streaming()` mit einem Dictionary aufrufen, das `request_id` mit der Antwort des Benutzers verknüpft

#### 2. **Streaming-Ausführungsmuster**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Ereignisgesteuerte Architektur**
Hören Sie auf spezifische Ereignisse, um den Workflow zu steuern:
- `RequestInfoEvent` - Menschliche Eingabe wird benötigt (Workflow pausiert)
- `WorkflowOutputEvent` - Endergebnis ist verfügbar (Workflow abgeschlossen)
- `WorkflowStatusEvent` - Statusänderungen (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS usw.)

#### 4. **Benutzerdefinierte Executors mit @handler**
Der `DecisionManager` zeigt, wie man Executors erstellt, die:
- Mit dem `@handler`-Dekorator Methoden als Workflow-Schritte bereitstellen
- Typisierte Nachrichten empfangen (z. B. `RequestResponse[HumanFeedbackRequest, str]`)
- Workflow durch das Senden von Nachrichten an andere Executors steuern
- Zugriff auf den Kontext über `WorkflowContext` haben

#### 5. **Bedingte Steuerung mit menschlichen Entscheidungen**
Sie können Bedingungsfunktionen erstellen, die menschliche Antworten auswerten:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Anwendungen in der Praxis:

1. **Genehmigungs-Workflows**
   - Managergenehmigung vor der Bearbeitung von Spesenabrechnungen einholen
   - Menschliche Überprüfung vor dem Versenden automatisierter E-Mails verlangen
   - Hochwertige Transaktionen vor der Ausführung bestätigen

2. **Inhaltsmoderation**
   - Fragwürdige Inhalte zur menschlichen Überprüfung kennzeichnen
   - Moderatoren die endgültige Entscheidung bei Grenzfällen treffen lassen
   - Menschen einbeziehen, wenn die KI-Genauigkeit niedrig ist

3. **Kundendienst**
   - KI routinemäßige Fragen automatisch bearbeiten lassen
   - Komplexe Probleme an menschliche Agenten eskalieren
   - Kunden fragen, ob sie mit einem Menschen sprechen möchten

4. **Datenverarbeitung**
   - Menschen bitten, mehrdeutige Dateneinträge zu klären
   - KI-Interpretationen unklarer Dokumente bestätigen
   - Benutzern die Wahl zwischen mehreren gültigen Interpretationen überlassen

5. **Sicherheitskritische Systeme**
   - Menschliche Bestätigung vor irreversiblen Aktionen verlangen
   - Genehmigung vor dem Zugriff auf sensible Daten einholen
   - Entscheidungen in regulierten Branchen (Gesundheitswesen, Finanzen) bestätigen

6. **Interaktive Agenten**
   - Konversationsbots erstellen, die Folgefragen stellen
   - Assistenten entwickeln, die Benutzer durch komplexe Prozesse führen
   - Agenten entwerfen, die Schritt für Schritt mit Menschen zusammenarbeiten

### 🔄 Vergleich: Mit vs. Ohne Mensch-in-der-Schleife

| Funktion | Bedingter Workflow | Mensch-in-der-Schleife-Workflow |
|----------|--------------------|---------------------------------|
| **Ausführung** | Einzelnes `workflow.run()` | Schleife mit `run_stream()` + `send_responses_streaming()` |
| **Benutzereingabe** | Keine (vollständig automatisiert) | Interaktive Eingabeaufforderungen über `input()` oder UI |
| **Komponenten** | Agenten + Executors | + RequestInfoExecutor + DecisionManager |
| **Ereignisse** | Nur AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent usw. |
| **Pausieren** | Kein Pausieren | Workflow pausiert bei RequestInfoExecutor |
| **Menschliche Kontrolle** | Keine menschliche Kontrolle | Menschen treffen wichtige Entscheidungen |
| **Anwendungsfall** | Automatisierung | Zusammenarbeit & Überwachung |

### 🚀 Erweiterte Muster:

#### Mehrere menschliche Entscheidungspunkte
Sie können mehrere `RequestInfoExecutor`-Knoten im selben Workflow haben:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout-Verarbeitung
Timeouts für menschliche Antworten implementieren:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integration mit Rich UI
Anstelle von `input()` Integration mit Web-UI, Slack, Teams usw.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Bedingte Mensch-in-der-Schleife
Nur in bestimmten Situationen menschliche Eingaben anfordern:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Best Practices:

1. **RequestInfoMessage immer unterklassen**
   - Bietet Typensicherheit und Validierung
   - Ermöglicht reichhaltigen Kontext für die UI-Darstellung
   - Klärt die Absicht jedes Anforderungstyps

2. **Beschreibende Eingabeaufforderungen verwenden**
   - Kontext zu Ihrer Anfrage angeben
   - Konsequenzen jeder Wahl erklären
   - Fragen einfach und klar halten

3. **Unerwartete Eingaben behandeln**
   - Benutzerantworten validieren
   - Standardwerte für ungültige Eingaben bereitstellen
   - Klare Fehlermeldungen geben

4. **Anfrage-IDs verfolgen**
   - Die Korrelation zwischen request_id und Antworten nutzen
   - Den Zustand nicht manuell verwalten

5. **Nicht blockierend gestalten**
   - Threads nicht blockieren, während auf Eingaben gewartet wird
   - Überall asynchrone Muster verwenden
   - Gleichzeitige Workflow-Instanzen unterstützen

### 📚 Verwandte Konzepte:

- **Agent Middleware** - Agentenaufrufe abfangen (vorheriges Notebook)
- **Workflow-Zustandsverwaltung** - Workflow-Zustand zwischen Ausführungen speichern
- **Multi-Agenten-Kollaboration** - Mensch-in-der-Schleife mit Agententeams kombinieren
- **Ereignisgesteuerte Architekturen** - Reaktive Systeme mit Ereignissen erstellen

---

### 🎓 Herzlichen Glückwunsch!

Sie haben Mensch-in-der-Schleife-Workflows mit Microsoft Agent Framework gemeistert! Sie wissen jetzt, wie man:
- ✅ Workflows pausiert, um menschliche Eingaben zu sammeln
- ✅ RequestInfoExecutor und RequestInfoMessage verwendet
- ✅ Streaming-Ausführung mit Ereignissen handhabt
- ✅ Benutzerdefinierte Executors mit @handler erstellt
- ✅ Workflows basierend auf menschlichen Entscheidungen steuert
- ✅ Interaktive KI-Agenten entwickelt, die mit Menschen zusammenarbeiten

**Dies ist ein entscheidendes Muster für den Aufbau vertrauenswürdiger, kontrollierbarer KI-Systeme!** 🚀



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
